In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("/Users/bytedance/MLTrader-deploy/")
import sys
sys.path.insert(0, ".")
import research.orderbook_strategies.utils.helper as helper
import research.orderbook_strategies.utils.stats as stats
import research.orderbook_strategies.utils.factor_analysis as factor_analysis
import research.orderbook_strategies.utils.factors as factors
import dask
from dask import compute, delayed
from dask.diagnostics import ProgressBar
from imp import reload   
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from decimal import Decimal, getcontext
from research.data_binance.utils.binance_util import get_um_asset_info
import zipfile
from pathlib import Path
import pandas as pd
import csv
from concurrent.futures import ThreadPoolExecutor
import io


ENVIRONMENT_NAME=ENV.LOCAL
[INFO] ASSET_INFO updated


In [3]:
import warnings
 
warnings.filterwarnings('ignore')

product_list = ["ETHUSDT", "OPUSDT", "BTCUSDT", "APTUSDT", "1000PEPEUSDT"]

In [4]:
# product = "ETHUSDT"
# error_file = Path("/Volumes/AdamDrive/binance_data/data/futures/um/daily/aggTrades") / product / "ETHUSDT-aggTrades-2023-03-21.csv"
for product in product_list[4:]:
    p = Path("/Volumes/AdamDrive/binance_data/data/futures/um/daily/trades") / product
    p2 = Path("/Volumes/AdamDrive/binance_data/data/futures/um/daily/aggTrades") / product
    os.system(f"bash /Users/bytedance/MLTrader-deploy/research/data_binance/utils/unzip_all.sh {str(p2)}")
    error_aggTrades_files = {}
    file_paths = []
    workers = ThreadPoolExecutor(8)
    
    def procee_file(file_path):
        with open(file_path, "r") as f:
            reader = csv.reader(f)
            for row in reader:
                if row[0] == "agg_trade_id":
                    continue
                agg_trade_id,price,quantity,first_trade_id,last_trade_id,transact_time,is_buyer_maker = row
                if float(quantity) <= 0 or float(price) <= 0:
                    # error
                    error_aggTrades_files[file_path.name] = file_path
                    print("error in file: ", str(file_path.name))
                    break
    for file_path in (p2).glob("*.csv"):
        workers.submit(procee_file, file_path) 
    workers.shutdown()
    
    precision = get_um_asset_info(product).quantityPrecision
    if precision != 0:
        getcontext().prec = precision
    for error_file in error_aggTrades_files.values():
        with open(error_file, "r") as f:
            reader = csv.reader(f)
            rows = list(reader)
        print("fixing {}".format(error_file.name))
        trade_file_path = Path("/Volumes/AdamDrive/binance_data/data/futures/um/daily/trades") / product / error_file.name.replace("aggTrades", "trades")
        trade_file_path = trade_file_path.with_suffix(".zip")
        with zipfile.ZipFile(trade_file_path, 'r') as z:
            with z.open(z.infolist()[0].filename, 'r') as csvfile:
                csvreader = csv.reader(io.TextIOWrapper(csvfile, newline=''))
                subrow = None
                for i, row in tqdm(enumerate(rows)):
                    if row[0] == "agg_trade_id":
                        continue
                    agg_trade_id,price,quantity,first_trade_id,last_trade_id,transact_time,is_buyer_maker = row
                    if precision == 0:
                        new_qty = 0
                    else:
                        new_qty = Decimal("0.")
                    new_time = None
                    while True:
                        try:
                            if subrow is None:
                                subrow = next(csvreader)
                            if subrow[0] == "id":
                                subrow = None
                                continue
                            id, price, qty, quote_qty, time, is_buyer_maker = subrow
                            if int(first_trade_id) <= int(id) <= int(last_trade_id):
                                if precision == 0:
                                    qty = int(qty)
                                else:
                                    qty = Decimal(qty)
                                new_qty += qty
                                new_time = time
                                subrow = None
                            else:
                                break
                        except StopIteration:
                            break
                        except Exception as e:
                            print(e)
                    assert new_qty > 0
                    rows[i][2] = str(new_qty)
                    rows[i][5] = new_time
                assert subrow is None
        with open(str(error_file), "w") as f:
            writer = csv.writer(f)
            writer.writerows(rows)
        print("done fixing ", error_file.name)
    os.system(f"rm -v {str(p2)}/*.zip")
    

unzip:  cannot find or open *.zip, *.zip.zip or *.zip.ZIP.

No zipfiles found.
rm: /Volumes/AdamDrive/binance_data/data/futures/um/daily/aggTrades/1000PEPEUSDT/*.zip: No such file or directory
